# SIR-X

This notebook exemplifies how Open-SIR can be used to fit the SIR-X model by [Maier and Dirk (2020)](https://science.sciencemag.org/content/early/2020/04/07/science.abb4557.full) to existing data and make predictions. The SIR-X model is an standard generalization of the Susceptible-Infectious-Removed (SIR) model, which includes the influence of exogeneous factors such as policy changes, lockdown of the whole population and quarantine of the infectious individuals.

To validate the Open-SIR implementation of the SIR-X model, it will be attempted to reproduce the parameter fitting published in the [suplementary material](https://science.sciencemag.org/cgi/content/full/science.abb4557/DC1) of the original recent article published by [Maier and Dirk (2020)](https://science.sciencemag.org/content/early/2020/04/07/science.abb4557.full). For simplicity, the validation will be performed only for the city of Guangdong, China.

## Import modules

In [ ]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt

## Data sourcing

We will source data from the repository of the [John Hopkins University COVID-19 dashboard] (https://coronavirus.jhu.edu/map.html) published formally as a correspondence in [The Lancet](https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30120-1/fulltext#seccestitle10). This time series data contains the number of reported cases $C(t)$ per day for a number of cities.



In [ ]:
# Source data from John Hokpins university reposotiry
jhu_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/who_covid_19_situation_reports/who_covid_19_sit_rep_time_series/who_covid_19_sit_rep_time_series.csv"
jhu_df = pd.read_csv(jhu_link)
# Explore the dataset
jhu_df.head(10)

It is observed that the column "Province/States" contains the name of the cities, and since the forth column a time series stamp (or index) is provided to record daily data of reported cases. Additionally, there are many days without recorded data for a number of chinese cities. This won't be an issue for parameter fitting as **Open-SIR** doesn't require uniform spacement of the observed data.

### Data preparation

In the following lines, the time series for Guangdong reported cases $C(t)$ is extracted from the original dataframe. Thereafter, the columns are converted to a pandas date time index in order to perform further data preparation steps.

In [ ]:
China = jhu_df[jhu_df[jhu_df.columns[1]]=="China"]
city_name = "Guangdong"
city = China[China["Province/States"] == city_name]
city = city.drop(columns=["Province/States", "Country/Region", "WHO region",])
time_index = pd.to_datetime(city.columns)
data = city.values
# Visualize the time
ts = pd.Series(data = city.values[0], index = time_index)

Using the function ts.plot() a quick visualization of the dataset is obtained:

In [ ]:
ts.plot()
plt.show()

Data cleaning

In [ ]:
ts_clean = ts.dropna()
# Extract data
ts_fit = ts_clean['2020-01-21':"2020-02-12"]
# Convert index to numeric
ts_num = pd.to_numeric(ts_fit.index)
t0 = ts_num[0]
# Convert datetime to days
t_days = (ts_num-t0)/(10**9*86400)
t_days = t_days.astype(int).values
# t_days is an input for SIR

In [ ]:
# Define the X number
nX = ts_fit.values # Number of infected
N = 104.3e6 # Population size of Guangdong

Exploration of the dataset

In [ ]:
ts_fit.plot(style='ro')
plt.xlabel("Number of infected")
plt.show()

The missing data between the 25th of January and the 31st of January doesn't prevent to fit the SIR-X model

### Setting up SIR and SIR-X models

The population $N$ of the city is a necessary input for the model. In this notebook, this was hardocded, but it can be sourced directly from a web source.

Note that whilst the SIR model estimates directly the number of infected people, $N I(t)$, SIR-X estimates the number of infected people based on the number of tested cases that are in quarantine or in an hospital $N X(t)$

In [ ]:
# These lines are required only if opensir wasn't installed using pip install, or if opensir is running in the pipenv virtual environment
import sys
path_opensir = '../../'
sys.path.append(path_opensir)

# Import SIR and SIRX models
from opensir.models import SIR, SIRX
nX = ts_fit.values # Number of observed infections of the time series
N = 104.3e6 # Population size of Guangdong
params = [0.95, 0.38]
w0 = (N-nX[0], nX[0], 0)

G_sir = SIR()
G_sir.set_params(p=params, initial_conds=w0)
G_sir.fit_input=2
G_sir.fit(t_days, nX, N)
G_sir.solve(t_days[-1], t_days[-1]+1)
t_SIR = G_sir.fetch()[:,0]
I_SIR = G_sir.fetch()[:,2]

In [ ]:
plt.plot(t_SIR, I_SIR)
plt.plot(t_days, nX, 'ro')
plt.show()

The SIR model is clearly not appropriate to fit this data. We will repeat the process with a SIR-X model

In [ ]:
g_sirx = SIRX()
params = [6.2/8, 1/8, 0.05, 0.05, 5]
# X_0 can be directly ontained from the statistics
n_x0 = nX[0]            # Number of people tested positive
n_i0 = nX[0]

w0 = (N-n_x0-n_i0, n_i0, 0, n_x0)
g_sirx.set_params(p=params, initial_conds=w0)
# Fit all parameters
fit_index=[False, False, True, True, True]
g_sirx.fit(t_days, nX, N, fit_index = fit_index)
g_sirx.solve(t_days[-1], t_days[-1]+1)
t_sirx = g_sirx.fetch()[:,0]
inf_sirx = g_sirx.fetch()[:,4]

In [ ]:
g_sirx.p

In [ ]:
plt.plot(t_sirx, inf_sirx)
plt.plot(t_SIR, I_SIR)
plt.plot(t_days, nX, 'ro')
plt.legend(["SIR-X", "SIR", "observed"])
plt.title("Guangdong")
plt.show()

After fitting the parameters, the effective infectious period $T_{I,eff}$ and the effective reproduction rate $R_{0,eff}$ can be obtained from the model properties

$$ T_{I,eff} = (\beta + \kappa + \kappa_0)^{-1} $$
$$ R_{0,eff} = \alpha T_{I,eff}$$

Aditionally, the Public containment leverage $P$ and the quarantine probability $Q$ can be calculated through:

$$ P = \frac{\kappa_0}{\kappa_0 + \kappa} $$
$$ Q = \frac{\kappa_0 + \kappa}{\beta + \kappa_0 + \kappa} $$

In [ ]:
print("Effective infectious period T_I_eff =  %.2f days " % g_sirx.t_inf_eff )
print("Effective reproduction rate R_0_eff =  %.2f, Maier and Brockmann = %.2f" % (g_sirx.r0_eff, 3.02))
print("Public containment leverage =  %.2f, Maier and Brockmann = %.2f" % (g_sirx.pcl, 0.75))
print("Quarantine probability =  %.2f, Maier and Brockmann = %.2f" % (g_sirx.q_prob, 0.51))

Now we can use the model to predict when the peak will occur and what will be the maximum number of infected

In [ ]:
# Predict
g_sirx.solve(40,41)
# Plot
plt.plot(g_sirx.fetch()[:,4]) # X(t)
plt.plot(g_sirx.fetch()[:,2]) # I(t)
plt.xlabel('Day')
plt.ylabel('Number of people')
plt.legend(["X(t): tested and quarantined","I(t) = infected"])
plt.title(city_name)
plt.show()

The model was trained with a limited amount of data. It is clear to observe that since the measures took place in Guangdong, at least 6 weels pf quarantine were necessary to control the pandemics. Note that a limitation of this model is that it predicts an equilibrium where the number of infected is 0 after a short time. In reality, this amount will decrease to a small number. What we see in the TV is the X(t) curve. After the curve "flattens", it is necessary to keep quarantine for more time and perform effective contact tracing of the remainder of the infected people who hasn't recovered yet.